In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.plotting_3d import plot_event


In [14]:
all_df = pd.read_pickle("data_files/all_df.pkl")


In [15]:
all_df.columns

Index(['wc_nue_score', 'wc_numu_score', 'wc_nc_delta_score', 'wc_nc_pio_score',
       'wc_numu_cc_flag', 'wc_run', 'wc_subrun', 'wc_event', 'wc_match_isFC',
       'wc_truth_nuEnergy',
       ...
       'proton_blip_x', 'proton_blip_y', 'proton_blip_z', 'proton_blip_energy',
       'other_blip_x', 'other_blip_y', 'other_blip_z', 'other_blip_energy',
       'pelee_shr_energy', 'net_weight'],
      dtype='object', length=105)

In [16]:
# event with a proton blip upstream of one shower
event_index = 7

# each entry is name : (points, color, cmap, size, visible)
points_dic = {}

points_dic["WC_reco_nu_vtx"] = (
    np.array([all_df["wc_reco_nuvtxX"].to_numpy()[event_index], all_df["wc_reco_nuvtxY"].to_numpy()[event_index], all_df["wc_reco_nuvtxZ"].to_numpy()[event_index]]),
    "green",
    None,
    10,
    "legendonly"
)
points_dic["true_nu_vtx"] = (
    np.array([all_df["wc_truth_vtxX"].to_numpy()[event_index], all_df["wc_truth_vtxY"].to_numpy()[event_index], all_df["wc_truth_vtxZ"].to_numpy()[event_index]]),
    "red",
    None,
    10,
    "legendonly"
)
points_dic["true_corr_nu_vtx"] = (
    np.array([all_df["wc_truth_corr_nuvtxX"].to_numpy()[event_index], all_df["wc_truth_corr_nuvtxY"].to_numpy()[event_index], all_df["wc_truth_corr_nuvtxZ"].to_numpy()[event_index]]),
    "yellow",
    None,
    10,
    "legendonly"
)
points_dic["Trec_spacepoints"] = (
    np.vstack((all_df["wc_Trec_spacepoints_x"].to_numpy()[event_index], all_df["wc_Trec_spacepoints_y"].to_numpy()[event_index], all_df["wc_Trec_spacepoints_z"].to_numpy()[event_index])).T,
    "red",
    None,
    1,
    "legendonly"
)
points_dic["Treccharge_spacepoints"] = (
    np.vstack((all_df["wc_Treccharge_spacepoints_x"].to_numpy()[event_index], all_df["wc_Treccharge_spacepoints_y"].to_numpy()[event_index], all_df["wc_Treccharge_spacepoints_z"].to_numpy()[event_index])).T,
    "blue",
    None,
    1,
    "legendonly"
)
points_dic["Trecchargeblob_spacepoints"] = (
    np.vstack((all_df["wc_Trecchargeblob_spacepoints_x"].to_numpy()[event_index], all_df["wc_Trecchargeblob_spacepoints_y"].to_numpy()[event_index], all_df["wc_Trecchargeblob_spacepoints_z"].to_numpy()[event_index])).T,
    all_df["wc_Trecchargeblob_spacepoints_q"].to_numpy()[event_index],
    "jet",
    1,
    True
)

points_dic["electron blips"] = (
    np.vstack((all_df["electron_blip_x"].to_numpy()[event_index], all_df["electron_blip_y"].to_numpy()[event_index], all_df["electron_blip_z"].to_numpy()[event_index])).T,
    "orange",
    None,
    5,
    True
)
points_dic["proton blips"] = (
    np.vstack((all_df["proton_blip_x"].to_numpy()[event_index], all_df["proton_blip_y"].to_numpy()[event_index], all_df["proton_blip_z"].to_numpy()[event_index])).T,
    "blue",
    None,
    5,
    True
)
points_dic["other blips"] = (
    np.vstack((all_df["other_blip_x"].to_numpy()[event_index], all_df["other_blip_y"].to_numpy()[event_index], all_df["other_blip_z"].to_numpy()[event_index])).T,
    "purple",
    None,
    5,
    True
)

plot_event(event_index, points_dic)
